In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
from pyFTS.common import Util
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import math
import statistics
from sklearn.preprocessing import StandardScaler

import sys
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")

from embfts.util.DataSetUtil import DataSetUtil
from embfts.util.StatisticsUtil import StatisticsUtil

In [3]:
data_set_util = DataSetUtil()
statistics_util = StatisticsUtil()

## Data set Kaggle Smart Home

In [4]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/HomeC.csv', sep=',')
df = df.drop(labels=['icon','summary','cloudCover', 'House overall [kW]', 'time'], axis=1)
df = df.loc[0: : 10]
data = data_set_util.clean_dataset(df)
data = data_set_util.series_to_supervised_mimo(data, 1, 1)
data

/home/hugo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,use [kW](t-1),gen [kW](t-1),Dishwasher [kW](t-1),Furnace 1 [kW](t-1),Furnace 2 [kW](t-1),Home office [kW](t-1),Fridge [kW](t-1),Wine cellar [kW](t-1),Garage door [kW](t-1),Kitchen 12 [kW](t-1),...,temperature(t),humidity(t),visibility(t),apparentTemperature(t),pressure(t),windSpeed(t),windBearing(t),precipIntensity(t),dewPoint(t),precipProbability(t)
10,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
20,1.585083,0.003417,0.000050,0.022100,0.678733,0.440200,0.121450,0.007433,0.013583,0.000350,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
30,0.533817,0.003450,0.000000,0.020633,0.062967,0.270033,0.004950,0.007033,0.013117,0.000750,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
40,1.190950,0.003083,0.000217,0.387400,0.061883,0.256867,0.004933,0.101783,0.012533,0.000700,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
50,1.582083,0.003067,0.000050,0.022050,0.681717,0.242433,0.005550,0.122400,0.013550,0.000567,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503860,1.068350,0.004183,0.000100,0.021550,0.639333,0.041650,0.005350,0.020400,0.013450,0.000567,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503870,1.621417,0.004183,0.000033,0.441950,0.645467,0.041767,0.129317,0.008067,0.012883,0.000317,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503880,0.896250,0.004150,0.000233,0.496633,0.063000,0.042317,0.123800,0.007850,0.012433,0.000467,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503890,1.311617,0.004167,0.000017,0.359167,0.645433,0.041917,0.005133,0.008167,0.013083,0.000650,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51


## Persistence method - Naive forecasting

In [5]:
# # Create a lag feature
# var = pd.DataFrame(y_appliance)
# dataframe = pd.concat([var.shift(1), var], axis=1)
# dataframe.columns = ['t', 't+1']
# dataframe

In [6]:
# Create a baseline model (Naive model)
def model_persistence(x):
  return x

def test_persistence(test_X):
    predictions = list()
    for x in test_X:
        yhat = model_persistence(x)
        predictions.append(yhat)
    return predictions

## Sliding Window PyFTS:Persistence

In [9]:
def sliding_window(data,n_windows,train_size):

    result = {
        "window": [],
        "rmse": [],
        "mape": [],
        "mae": [],
        "r2": [],
        "variable":[]
    }
    
    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, 0.75, inc=1):
        if len(ttest) > 0:

            print('-' * 20)
            print(f'training window {(ct)}')

            Xtrain = ttrain.loc[:,'use [kW](t-1)':'precipProbability(t-1)']
            ytrain = ttrain.loc[:,'use [kW](t)':'precipProbability(t)']
            Xtest = ttest.loc[:,'use [kW](t-1)':'precipProbability(t-1)']
            ytest = ttest.loc[:,'use [kW](t)':'precipProbability(t)']

            test_X = pd.DataFrame(Xtest.values,columns = list(df.columns))
            test_y = pd.DataFrame(ytest.values,columns = list(df.columns))
            
            for col in test_y.columns:
                forecast = test_persistence(test_X[col])
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(test_y[col], label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()

                #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics")
                rmse = Measures.rmse(test_y[col],forecast)
                mape = Measures.mape(test_y[col],forecast)
                mae = mean_absolute_error(test_y[col], forecast)
                r2 = r2_score(test_y[col], forecast)

                result["rmse"].append(rmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["window"].append(ct)
                result["variable"].append(col)

    measures = pd.DataFrame(result)
    return measures

In [10]:
naive_result =  sliding_window(data=data,n_windows=30,train_size=0.75)

--------------------
training window 0
--------------------
training window 1679
--------------------
training window 3358
--------------------
training window 5037
--------------------
training window 6716
--------------------
training window 8395
--------------------
training window 10074
--------------------
training window 11753
--------------------
training window 13432
--------------------
training window 15111


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 16790
--------------------
training window 18469
--------------------
training window 20148
--------------------
training window 21827
--------------------
training window 23506
--------------------
training window 25185
--------------------
training window 26864
--------------------
training window 28543
--------------------
training window 30222
--------------------
training window 31901
--------------------
training window 33580
--------------------
training window 35259
--------------------
training window 36938
--------------------
training window 38617
--------------------
training window 40296
--------------------
training window 41975
--------------------
training window 43654


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 45333
--------------------
training window 47012
--------------------
training window 48691


In [11]:
columns = list(df.columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": []
}

measures = naive_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures = round(pd.DataFrame(final_result),3) 



In [12]:
final_measures

,variable,rmse,mae,mape,r2
0,use [kW],0.846,0.468,251.684,-0.100
1,gen [kW],0.040,0.015,inf,0.842
2,Dishwasher [kW],0.171,0.027,inf,0.071
3,Furnace 1 [kW],0.158,0.072,135.154,-0.475
4,Furnace 2 [kW],0.184,0.096,70.933,-0.518
5,Home office [kW],0.045,0.010,8.728,0.690
6,Fridge [kW],0.073,0.033,256.398,0.066
7,Wine cellar [kW],0.029,0.009,40.135,0.651
8,Garage door [kW],0.012,0.002,9.496,-0.857
9,Kitchen 12 [kW],0.009,0.001,inf,-0.135
